# Some powerline can be disconnected, and there is ~~nothing~~ something you can do about it
Try me out interactively with: [![Binder](./img/badge_logo.svg)](https://mybinder.org/v2/gh/rte-france/Grid2Op/master)

To model the reality of powersystem operation and planning grid2op offer the possibility, when setting up an environment, to incldued powerline disconnection.

This is a really important part of power system decision process. Indeed, when a powerline is being built it is often for more than a few decades (in some TSO some equipements have been built and are still in service almost 100 years ago!). The **maintenance** of such equipements is then an important feature to model properly. In case of grid2op, the **maitnenance** is modeled as a disconnection of a powerline and the impossibility to reconnect it for a given amount of time. It is a planned operation for which you have some information in the observation:
- [`obs.time_next_maintenance`](https://grid2op.readthedocs.io/en/master/observation.html#grid2op.Observation.BaseObservation.time_next_maintenance) will give you, for each powerline, in how many time steps this powerline will be disconnected of the grid
- [`obs.duration_next_maintenance`](https://grid2op.readthedocs.io/en/master/observation.html#grid2op.Observation.BaseObservation.duration_next_maintenance) will give you, for each powerline, for how long (in number of time steps) this powerline will stay disconnected

On the other important property of the powergrid is that they cover large areas, usually they are the side of a whole country / state for example, counting hundred of thousands of "equipment" (varying in size from the tiny screws to fix two things together to wide transformers of the size of a building...). The probability, in these conditions, that any of this equipment fails is far from neglectable. For the record, the "failure" of an equipment have dozens of possible causes: it is in a bad shape (not enough maintenance), it suffers from an natural external aggression (wind storm, struck by lightning bolts, a tree falls on it, the external temperatur is too hot causing some thermal issues etc.) be the victim of malicious attack (a person hack a piece of software or physically attack the equipement with a bulldozer or a bombing for example) or it is not operated outside of its standard operation range (human error, bad data are send to the control center etc.) or any other cause you can imagine. Most of these phenomenon have a common consequence: one or more powerlines will be disconnected from the grid. This is what we call **hazards** in the grid2op framework. Compared to maintenance, **hazards** are not predictable (nothing, before it happens, indicates that there will be such hazards in the next few time steps).

Actually, in our framework, we also added the capability to explicitely model attacks on the grid. We call this feature **Opponent**. For now, we model the opponent as an entity that can take, at each time step, the decision to disconnect a powerline. It has the same effect as the **hazards** (you know what it will do before it is happening) but is different in the following sense:

- **hazards** will refer to random non predictable events (let's call it *natural failures*), they are, by essence, random
- **attack** (the action performed by an Opponent) will refer to possibly adversarial attacks, targeting equipement (not necessarily at random).


In both cases, you know when a **maintenance**, a **hazard** or an **attack** is happening if you look at the [`obs.time_before_cooldown_line`](https://grid2op.readthedocs.io/en/master/observation.html#grid2op.Observation.BaseObservation.time_before_cooldown_line) vector. It gives you, for each powerline, in how many time step at **minimum** you need to wait before being able to reconnect a powerline. If this is 0 you can change its status. If this > 0 you need to wait the appropriate number of time steps.

**NB** whether it's an attack, a hazard or a maintenance, the powerline will be automatically *disconnected*. They will stay *disconnected* until you reconnect them. This entails that if you don't reconnect them, they will remain disconnected.

# How to spot a maintenance

In [ ]:
import grid2op
import numpy as np
print("grid2op version: {}".format(grid2op.__version__))
env = grid2op.make("l2rpn_wcci_2020", difficulty="0")  # TODO make a test env for that

In [ ]:
env.seed(0)  # to get reproducible results
env.set_id(0)  # indicate to always read the same chronics, the first one
obs = env.reset()
do_nothing_action = env.action_space()
line_id = 56
start_maintenance = 1259  # first maintenance at 1260 (number of time step) then at 2412
reconnect_action = env.action_space({"set_line_status": [(line_id, +1)]})
# we know (because we made it) that there will be maintenance at ... so we "fast forward until that point"
env.fast_forward_chronics(start_maintenance)
obs = env.get_obs()
print("Can i act on the pwoerline: {}".format(obs.time_before_cooldown_line[line_id] == 0))
print("Is the powerline connected: {}".format(obs.line_status[line_id]))

In [ ]:
# one step pass
obs, reward, done, info = env.step(do_nothing_action)
print("Can i act on the pwoerline: {}".format(obs.time_before_cooldown_line[line_id] == 0))
print("In how many time I will be able to reconnect it: {}".format(obs.time_before_cooldown_line[line_id]))
print("Is the powerline connected: {}".format(obs.line_status[line_id]))

In [ ]:
# let's try to reconnect the powerline
obs, reward, done, info = env.step(reconnect_action)
print("Can i act on the pwoerline: {}".format(obs.time_before_cooldown_line[line_id] == 0))
print("In how many time I will be able to reconnect it: {}".format(obs.time_before_cooldown_line[line_id]))
print("Is the powerline connected: {}".format(obs.line_status[line_id]))
# it has no effect :-(

In [ ]:
# let's then do nothing for 95 remaining time step
# (i can reconnect it only in 95 time steps as said above -> if i try before that it fails...)
for _ in range(95):
    obs, reward, done, info = env.step(do_nothing_action)
print("Can i act on the pwoerline: {}".format(obs.time_before_cooldown_line[line_id] == 0))
print("In how many time I will be able to reconnect it: {}".format(obs.time_before_cooldown_line[line_id]))
print("Is the powerline connected: {}".format(obs.line_status[line_id]))
print("The flow on it is {:.1f}A (of course because it is switched off)".format(obs.a_or[line_id]))

In [ ]:
# and now reconnect it
obs, reward, done, info = env.step(reconnect_action)
print("Can i act on the powerline: {}".format(obs.time_before_cooldown_line[line_id] == 0))
print("In how many time I will be able to reconnect it: {}".format(obs.time_before_cooldown_line[line_id]))
print("Is the powerline connected: {}".format(obs.line_status[line_id]))
print("The flow on it is {:.1f}A".format(obs.a_or[line_id]))